# Extrating Data From an Encryptrd  PDF. 

# Install packages

In [ ]:
!pip install PyPDF2==2.7
!pip install camelot-py[cv]

You should consider upgrading via the 'C:\Users\Ricmwas\Documents\python_projects\openvision-master\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import PyPDF2
import camelot
import pandas as pd

In [ ]:
def extract_table_from_protected_pdf(pdf_path, password):
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            # Create a PdfReader object
            reader = PyPDF2.PdfReader(file)
            
            # Check if the PDF is encrypted
            if reader.is_encrypted:
                # Decrypt the PDF
                if not reader.decrypt(password):
                    raise ValueError("Wrong password or unable to decrypt the PDF.")
            
            # Create a temporary decrypted PDF file
            decrypted_pdf_path = "decrypted_temp.pdf"
            with open(decrypted_pdf_path, 'wb') as decrypted_file:
                writer = PyPDF2.PdfWriter()
                for page in reader.pages:
                    writer.add_page(page)
                writer.write(decrypted_file)
            
            # Extract table using Camelot
            tables = camelot.read_pdf(decrypted_pdf_path, pages="all", flavor = 'stream')
            
            return tables
    
    except Exception as e:
        print("An error occurred:", e)

# Usage example
pdf_path = "Account_Statement.pdf"
password = '6595270'
tabl= extract_table_from_protected_pdf(pdf_path, password)


In [18]:
tabl[0].df.head()

,0,1,2,3,4,5,6,7
0,60100,,,,,,,
1,KENYA,,,,,,,
2,Trans,Details,Channel,Value,Reference,Deposit,Withdraw,Book Balance
3,10-8-2021,00030822,BFTC,10-8-2021,117b2eb336c,0.00,0.00,null
4,10-8-2021,00030821,BFTC,10-8-2021,117b2eb336c,0.00,0.00,null


In [ ]:

def process_table_to_dataframe(raw_data):
    """
    Process a single table to extract rows starting from the header ("Trans").
    """
    try:
        # Find the index of the row containing "Trans" (this will be your header)
        header_index_list = raw_data.index[raw_data[0] == "Trans"].tolist()
        if not header_index_list:
            # Skip if "Trans" is not found
            print("Header 'Trans' not found in the table. Skipping this table.")
            return None
        # Get the index of the header
        header_index = header_index_list[0]
        # Set the header row and extract the relevant data
        processed_data = pd.DataFrame(raw_data.values[header_index + 1:], columns=raw_data.iloc[header_index])
        # Clean column names
        processed_data.columns = processed_data.columns.str.strip()
        return processed_data
    
    except Exception as e:
        print(f"Error processing table: {e}")
        return None

def merge_tables(tabl):
    """
    Merge all tables in the list into one DataFrame.
    """
    merged_data = pd.DataFrame()
    for table in tabl:
        # Process each table
        processed_data = process_table_to_dataframe(table.df)
        if processed_data is not None:
            # Append to the merged DataFrame
            merged_data = pd.concat([merged_data, processed_data], ignore_index=True)
    return merged_data


Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
Header 'Trans' not found in the table. Skipping this table.
            Trans                              Details Channel       Value  \
0       10-8-2021                             00030822    BFTC   10-8-2021   
1       10-8-2021                             00030821    BFTC   10-8-2021   
2       10-8-2021                             05488667     SCH   10-8-2021   
3       10-8-2021                             05488669     SCH   10-8-2021   
4       10-8-2021                             05488671     SCH   10-8-2021   
...           ...                                  ...     ...         ...   
16069  24-11-2024     MPESA BANK C

In [ ]:
# Merge all tables
merged_data = merge_tables(tabl)
# Display the result
merged_data.tail()

,Trans,Details,Channel,Value,Reference,Deposit,Withdraw,Book Balance
16069,24-11-2024,MPESA BANK COMMISSION MPESA BANK,OMNI,24-11-2024,CB0147232,0.00,50.00,275.71 CR
16070,24-11-2024,EXCISE MPESA BANK COMMISSION,OMNI,24-11-2024,CB0147232,0.00,7.50,268.21 CR
16071,26-11-2024,AB646291CA24 HENRY MURIITHI KARANJA,OMNI,26-11-2024,CB0093762,"100,000.00",0.00,"100,268.21 CR"
16072,26-11-2024,CREDIT ALERT CRG CB0093762,OMNI,26-11-2024,CB0094134,0.00,35.00,"100,233.21 CR"
16073,26-11-2024,CREDIT CRG Excise Duty CB0093762,OMNI,26-11-2024,CB0094134,0.00,7.00,"100,226.21 CR"


In [25]:
merged_data.to_csv('Siz_Account_Statement.csv')